In [36]:
%pip install pandas fuzzywuzzy rapidfuzz nltk openpyxl jaro-winkler

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [60]:
import pandas as pd
from fuzzywuzzy import fuzz, process
from rapidfuzz import fuzz as rf_fuzz, process as rf_process
from rapidfuzz.distance import JaroWinkler as jw
import nltk
from difflib import SequenceMatcher
import jaro

## FuzzyWuzzy

In [ ]:
# Read CSV
df = pd.read_csv('clean_popcite.csv')

# Basic matching
def fuzzy_match(str1, str2):
    return fuzz.ratio(str1, str2)

# Find best match from a list
def find_best_match(query, choices):
    return process.extractOne(query, choices)

# Apply to DataFrame
df['match_score'] = df.apply(lambda x: fuzz.ratio(x['Authors'], x['Title']), axis=1)

In [7]:
df

,Cites,Authors,Title,Year,Source,Publisher,ArticleURL,CitesURL,GSRank,QueryDate,...,StartPage,EndPage,ECC,CitesPerYear,CitesPerAuthor,AuthorCount,Age,Abstract,FullTextURL,RelatedURL
0,12,"I Lewaa, MS Hafez, MA Ismail",Data integration using statistical matching te...,2021,Statistical Journal of the IAOS,content.iospress.com,https://content.iospress.com/articles/statisti...,https://scholar.google.com/scholar?cites=17311...,1,2025-05-26 10:45:08,...,NaN,NaN,12,3.00,4,3,4,… [2] presents a review for combining data fro...,NaN,https://scholar.google.com/scholar?q=related:1...
1,2,"M El Abassi, M Amnai, A Choukri, Y Fakhri…",Matching data detection for the integration sy...,2023,International Journal of …,academia.edu,https://www.academia.edu/download/95754849/162...,https://scholar.google.com/scholar?cites=61680...,2,2025-05-26 10:45:08,...,NaN,NaN,2,1.00,0,5,2,"… However, for data across multiple data sourc...",https://www.academia.edu/download/95754849/162...,https://scholar.google.com/scholar?q=related:j...
2,58,"J Yang, K Xian, P Wang, Y Zhang",A performance evaluation of correspondence gro...,2019,IEEE transactions on pattern …,ieeexplore.ieee.org,https://ieeexplore.ieee.org/abstract/document/...,https://scholar.google.com/scholar?cites=14537...,3,2025-05-26 10:45:08,...,NaN,NaN,58,9.67,15,4,6,"… 1, the subsequent procedure after correspond...",https://ieeexplore.ieee.org/iel7/34/4359286/08...,https://scholar.google.com/scholar?q=related:y...
3,43,"J Yang, S Quan, P Wang…",Evaluating local geometric feature representat...,2019,IEEE Transactions on …,ieeexplore.ieee.org,https://ieeexplore.ieee.org/abstract/document/...,https://scholar.google.com/scholar?cites=10653...,4,2025-05-26 10:45:08,...,NaN,NaN,43,7.17,11,4,6,Local geometric descriptors act as an essentia...,https://ieeexplore.ieee.org/iel7/83/4358840/08...,https://scholar.google.com/scholar?q=related:r...
4,22,"Y Zhu, J Yang",Automatic data matching for geospatial models:...,2019,Annals of GIS,Taylor &Francis,https://www.tandfonline.com/doi/abs/10.1080/19...,https://scholar.google.com/scholar?cites=18710...,5,2025-05-26 10:45:08,...,NaN,NaN,22,3.67,11,2,6,… Data similarity is the basis of automatic da...,https://www.tandfonline.com/doi/pdf/10.1080/19...,https://scholar.google.com/scholar?q=related:S...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2,"S Auzoux, B Ngaba, M Christina, B Heuclin, M R...",Experimental variables in sugarcane intercropp...,2023,Data in Brief,Elsevier,https://www.sciencedirect.com/science/article/...,https://scholar.google.com/scholar?cites=93023...,96,2025-05-26 10:45:08,...,NaN,NaN,2,1.00,0,5,2,"… Consequently, there is a need to standardize...",https://www.sciencedirect.com/science/article/...,https://scholar.google.com/scholar?q=related:r...
96,0,"A Coppens, V Maquil",Skeletal Data Matching and Merging from Multip...,2024,"… 2024, Valencia, Spain, September 15–18 …",books.google.com,https://books.google.com/books?hl=en&lr=&id=pX...,NaN,97,2025-05-26 10:45:08,...,NaN,NaN,0,0.00,0,2,1,… and led to ambiguities with the skeleton mat...,NaN,https://scholar.google.com/scholar?q=related:Q...
97,3,"A Kato, H Wakabayashi, M Bradford…",Accurate ground positioning obtained from 3d d...,2019,IGARSS 2019-2019 …,ieeexplore.ieee.org,https://ieeexplore.ieee.org/abstract/document/...,https://scholar.google.com/scholar?cites=15570...,98,2025-05-26 10:45:08,...,NaN,NaN,3,0.50,1,4,6,… and terrestrial 3D data is proposed in this ...,https://ieeexplore.ieee.org/iel7/8891871/88977...,https://scholar.google.com/scholar?q=related:n...
98,1,"UV Albrecht, D Lawin, S Kuhn…",Time bias awareness in ECG-Based multiple sour...,2022,Advances in Informatics …,ebooks.iospress.nl,https://ebooks.iospress.nl/volumearticle/60176,https://scholar.google.com/scholar?cites=54955...,99,2025-05-26 10:45:08,...,NaN,NaN,1,0.33,0,4,3,For cardiological datasets acquired via differ...,https://ebooks.iospress.nl/pdf/doi/10.3233/SHT...,https://sch

## RapidFuzz

In [10]:
# Same syntax as FuzzyWuzzy but faster
def rapid_match(str1, str2):
    return rf_fuzz.ratio(str1, str2)

# Best match
best_match = rf_process.extractOne("query", df['Authors'].tolist())

## NLTK 

In [11]:
from nltk.metrics.distance import jaro_winkler_similarity

def nltk_jaro_winkler(str1, str2):
    return jaro_winkler_similarity(str1, str2)

# Apply to CSV
df['jw_score'] = df.apply(lambda x: jaro_winkler_similarity(x['Authors'], x['Title']), axis=1)

## DiffLib

In [ ]:
def difflib_similarity(str1, str2):
    return SequenceMatcher(None, str1, str2).ratio()

df['difflib_score'] = df.apply(lambda x: difflib_similarity(x['Authors'], x['Title']), axis=1)

In [21]:
# Clean and prepare author data
df['Authors_clean'] = df['Authors'].fillna('').str.strip().str.lower()

# Remove empty authors
df_clean = df[df['Authors_clean'] != ''].copy()

print(f"Total rows with authors: {len(df_clean)}")

Total rows with authors: 100


## Compare

In [18]:
# Compare multiple algorithms
df['fuzzy_score'] = df.apply(lambda x: fuzz.ratio(x['Authors'], x['Authors']), axis=1)
df['rapid_score'] = df.apply(lambda x: rf_fuzz.ratio(x['Authors'], x['Authors']), axis=1)
df['jaro_winkler'] = df.apply(lambda x: jaro_winkler_similarity(x['Authors'], x['Authors']), axis=1)
# df['difflib_score'] = df.apply(lambda x: SequenceMatcher(None, x['name1'], x['name2']).ratio(), axis=1)

In [19]:
# Filter matches above threshold
matches = df[df['jaro_winkler'] > 0.2]

In [20]:
matches

,Cites,Authors,Title,Year,Source,Publisher,ArticleURL,CitesURL,GSRank,QueryDate,...,AuthorCount,Age,Abstract,FullTextURL,RelatedURL,match_score,jw_score,fuzzy_score,rapid_score,jaro_winkler
0,12,"I Lewaa, MS Hafez, MA Ismail",Data integration using statistical matching te...,2021,Statistical Journal of the IAOS,content.iospress.com,https://content.iospress.com/articles/statisti...,https://scholar.google.com/scholar?cites=17311...,1,2025-05-26 10:45:08,...,3,4,… [2] presents a review for combining data fro...,NaN,https://scholar.google.com/scholar?q=related:1...,7,0.434474,100,100.0,1.0
1,2,"M El Abassi, M Amnai, A Choukri, Y Fakhri…",Matching data detection for the integration sy...,2023,International Journal of …,academia.edu,https://www.academia.edu/download/95754849/162...,https://scholar.google.com/scholar?cites=61680...,2,2025-05-26 10:45:08,...,5,2,"… However, for data across multiple data sourc...",https://www.academia.edu/download/95754849/162...,https://scholar.google.com/scholar?q=related:j...,15,0.542857,100,100.0,1.0
2,58,"J Yang, K Xian, P Wang, Y Zhang",A performance evaluation of correspondence gro...,2019,IEEE transactions on pattern …,ieeexplore.ieee.org,https://ieeexplore.ieee.org/abstract/document/...,https://scholar.google.com/scholar?cites=14537...,3,2025-05-26 10:45:08,...,4,6,"… 1, the subsequent procedure after correspond...",https://ieeexplore.ieee.org/iel7/34/4359286/08...,https://scholar.google.com/scholar?q=related:y...,21,0.523559,100,100.0,1.0
3,43,"J Yang, S Quan, P Wang…",Evaluating local geometric feature representat...,2019,IEEE Transactions on …,ieeexplore.ieee.org,https://ieeexplore.ieee.org/abstract/document/...,https://scholar.google.com/scholar?cites=10653...,4,2025-05-26 10:45:08,...,4,6,Local geometric descriptors act as an essentia...,https://ieeexplore.ieee.org/iel7/83/4358840/08...,https://scholar.google.com/scholar?q=related:r...,26,0.477790,100,100.0,1.0
4,22,"Y Zhu, J Yang",Automatic data matching for geospatial models:...,2019,Annals of GIS,Taylor &Francis,https://www.tandfonline.com/doi/abs/10.1080/19...,https://scholar.google.com/scholar?cites=18710...,5,2025-05-26 10:45:08,...,2,6,… Data similarity is the basis of automatic da...,https://www.tandfonline.com/doi/pdf/10.1080/19...,https://scholar.google.com/scholar?q=related:S...,12,0.440128,100,100.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2,"S Auzoux, B Ngaba, M Christina, B Heuclin, M R...",Experimental variables in sugarcane intercropp...,2023,Data in Brief,Elsevier,https://www.sciencedirect.com/science/article/...,https://scholar.google.com/scholar?cites=93023...,96,2025-05-26 10:45:08,...,5,2,"… Consequently, there is a need to standardize...",https://www.sciencedirect.com/science/article/...,https://scholar.google.com/scholar?q=related:r...,22,0.556078,100,100.0,1.0
96,0,"A Coppens, V Maquil",Skeletal Data Matching and Merging from Multip...,2024,"… 2024, Valencia, Spain, September 15–18 …",books.google.com,https://books.google.com/books?hl=en&lr=&id=pX...,NaN,97,2025-05-26 10:45:08,...,2,1,… and led to ambiguities with the skeleton mat...,NaN,https://scholar.google.com/scholar?q=related:Q...,15,0.462535,100,100.0,1.0
97,3,"A Kato, H Wakabayashi, M Bradford…",Accurate ground positioning obtained from 3d d...,2019,IGARSS 2019-2019 …,ieeexplore.ieee.org,https://ieeexplore.ieee.org/abstract/document/...,https://scholar.google.com/scholar?cites=15570...,98,2025-05-26 10:45:08,...,4,6,… and terrestrial 3D data is proposed in this ...,https://ieeexplore.ieee.org/iel7/8891871/88977...,https://scholar.google.com/scholar?q=related:n...,25,0.539319,100,100.0,1.0
98,1,"UV Albrecht, D Lawin, S Kuhn…",Time bias awareness in ECG-Based multiple sour...,2022,Advances in Informatics …,ebooks.iospress.nl,https://ebooks.iospress.nl/volumearticle/60176,https://scholar.google.com/scholar?cites=54955...,99,2025-05-26 10:45:08,...,4,3,For cardiological datasets acquired via differ...,https:

In [64]:
# Method 1: Compare each author with every other author (pairwise comparison)
def compare_all_authors_pairwise():
    """Compare every author with every other author"""
    results = []
    
    for i, row1 in df_clean.iterrows():
        for j, row2 in df_clean.iterrows():
            if i < j:  # Avoid duplicate comparisons
                author1 = row1['Authors_clean']
                author2 = row2['Authors_clean']
                
                # Calculate similarities using different algorithms
                # fuzzy_score = fuzz.ratio(author1, author2)
                rapid_score = jw.normalized_distance(author1, author2) * 100
                jaro_winkler = jaro_winkler_similarity(author1, author2) * 100  # Convert to percentage
                jarolib = jaro.jaro_winkler_metric(author1, author2) * 100
                # difflib_score = SequenceMatcher(None, author1, author2).ratio()
                
                results.append({
                    'Index1': i,
                    'Index2': j,
                    'Author1': row1['Authors'],
                    'Author2': row2['Authors'],
                    # 'Title1': row1['Title'][:50] + '...',  # Truncate for readability
                    # 'Title2': row2['Title'][:50] + '...',
                    # 'FuzzyWuzzy_Score': fuzzy_score,
                    'RapidFuzz_Score': rapid_score,
                    'NLTK_Score': round(jaro_winkler, 2),
                    'Jaro_Winkler_Score': round(jarolib, 2),
                    # 'DiffLib_Score': round(difflib_score, 2)
                })
    
    return pd.DataFrame(results)

# Method 2: Find similar authors for each author (more efficient)
def find_similar_authors(threshold=80):
    """Find similar authors above a threshold"""
    results = []
    
    author_list = df_clean['Authors_clean'].tolist()
    
    for idx, author in enumerate(df_clean['Authors_clean']):
        # Use RapidFuzz for efficiency
        # rf_process.extract returns (match_string, score, match_index)
        matches = rf_process.extract(author, author_list, limit=5, score_cutoff=threshold)
        
        for match_result in matches:
            match_string, score, match_idx = match_result  # Unpack 3 values
            
            if idx != match_idx:  # Don't include self-matches
                results.append({
                    'Original_Index': df_clean.iloc[idx].name,
                    'Match_Index': df_clean.iloc[match_idx].name,
                    'Original_Author': df_clean.iloc[idx]['Authors'],
                    'Similar_Author': df_clean.iloc[match_idx]['Authors'],
                    'Similarity_Score': score,
                    'Original_Title': df_clean.iloc[idx]['Title'][:50] + '...',
                    'Similar_Title': df_clean.iloc[match_idx]['Title'][:50] + '...'
                })
    
    return pd.DataFrame(results).drop_duplicates()

# Method 3: Group similar authors
def group_similar_authors(threshold=85):
    """Group authors that are likely the same person"""
    similar_groups = []
    processed = set()
    
    for idx, row in df_clean.iterrows():
        if idx in processed:
            continue
            
        author = row['Authors_clean']
        # group = [{'index': idx, 'author': row['Authors'], 'title': row['Title']}]
        group = [{'index': idx, 'author': row['Authors']}]
        
        # Find all similar authors
        for idx2, row2 in df_clean.iterrows():
            if idx != idx2 and idx2 not in processed:
                similarity = rf_fuzz.ratio(author, row2['Authors_clean'])
                if similarity >= threshold:
                    group.append({'index': idx2, 'author': row2['Authors']})
                    processed.add(idx2)
        
        if len(group) > 1:  # Only keep groups with multiple similar authors
            similar_groups.append(group)
        
        processed.add(idx)
    
    return similar_groups


In [65]:
comparison = compare_all_authors_pairwise()

In [66]:
# compare to csv
comparison.to_csv('author_comparison.csv', index=False)

In [29]:
# Execute comparisons
print("1. Finding similar authors (threshold: 80%)...")
similar_authors = find_similar_authors(threshold=80)

print(f"Found {len(similar_authors)} potential similar author pairs")
print("\nTop 10 similar author pairs:")
print(similar_authors.head(10)[['Original_Author', 'Similar_Author', 'Similarity_Score']])

print("\n" + "="*50)
print("2. Grouping potentially same authors (threshold: 85%)...")
author_groups = group_similar_authors(threshold=85)

print(f"Found {len(author_groups)} groups of similar authors")
for i, group in enumerate(author_groups[:5]):  # Show first 5 groups
    print(f"\nGroup {i+1}:")
    for item in group:
        print(f"  - {item['author']} (Row {item['index']})")

# Save results
similar_authors.to_csv('similar_authors.csv', index=False)
# similar_authors.to_excel('similar_authors.xlsx', index=False)
print(f"\nResults saved to 'similar_authors.csv'")


1. Finding similar authors (threshold: 80%)...
Found 377 potential similar author pairs

Top 10 similar author pairs:
                              Original_Author  \
0                I Lewaa, MS Hafez, MA Ismail   
1  M El Abassi, M Amnai, A Choukri, Y Fakhri…   
2  M El Abassi, M Amnai, A Choukri, Y Fakhri…   
3  M El Abassi, M Amnai, A Choukri, Y Fakhri…   
4  M El Abassi, M Amnai, A Choukri, Y Fakhri…   
5             J Yang, K Xian, P Wang, Y Zhang   
6             J Yang, K Xian, P Wang, Y Zhang   
7             J Yang, K Xian, P Wang, Y Zhang   
8             J Yang, K Xian, P Wang, Y Zhang   
9                     J Yang, S Quan, P Wang…   

                                    Similar_Author  Similarity_Score  
0                                    W Jiang, J Ma              85.5  
1                                    Y Zhu, J Yang              85.5  
2                              T Hayashi, Y Ohsawa              85.5  
3                        B Engquist, K Ren, Y Yang        